In [18]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
df = pd.read_csv("10kmovies.csv")

In [3]:
df.head()

,id,title,release_date,genres,original_language,vote_average,vote_count,popularity,overview,budget,production_companies,revenue,runtime,tagline
0,758323,The Pope's Exorcist,2023-04-05,"['Horror', 'Mystery', 'Thriller']",English,7.4,619,5089.969,"Father Gabriele Amorth, Chief Exorcist of the ...",18000000,"['Screen Gems', '2.0 Entertainment', 'Jesus & ...",65675816,103,Inspired by the actual files of Father Gabriel...
1,640146,Ant-Man and the Wasp: Quantumania,2023-02-15,"['Action', 'Adventure', 'Science Fiction']",English,6.6,2294,4665.438,Super-Hero partners Scott Lang and Hope van Dy...,200000000,"['Marvel Studios', 'Kevin Feige Productions']",464566092,125,Witness the beginning of a new dynasty.
2,502356,The Super Mario Bros. Movie,2023-04-05,"['Animation', 'Adventure', 'Family', 'Fantasy'...",English,7.5,1861,3935.550,"While working underground to fix a water main,...",100000000,"['Universal Pictures', 'Illumination', 'Ninten...",1121048165,92,NaN
3,868759,Ghosted,2023-04-18,"['Action', 'Comedy', 'Romance']",English,7.2,652,2791.532,Salt-of-the-earth Cole falls head over heels f...,0,"['Skydance Media', 'Apple Studios']",0,120,Finding that special someone can be a real adv...
4,594767,Shazam! Fury of the Gods,2023-03-15,"['Action', 'Comedy', 'Fantasy', 'Adventure']",English,6.8,1510,2702.593,"Billy Batson and his foster siblings, who tran...",125000000,"['New Line Cinema', 'The Safran Company', 'DC ...",133437105,130,Oh. My. Gods.


In [4]:
df.shape

(10000, 14)

In [5]:
df.isnull().sum() #tagline,overview and release date have null values. Not interested in this so will drop them.

id                         0
title                      0
release_date              21
genres                     0
original_language          0
vote_average               0
vote_count                 0
popularity                 0
overview                  77
budget                     0
production_companies       0
revenue                    0
runtime                    0
tagline                 2759
dtype: int64

In [6]:
df.drop(columns = ['release_date','overview','tagline'], inplace = True) # Dropping the columns will null values.

In [7]:
df.head()

,id,title,genres,original_language,vote_average,vote_count,popularity,budget,production_companies,revenue,runtime
0,758323,The Pope's Exorcist,"['Horror', 'Mystery', 'Thriller']",English,7.4,619,5089.969,18000000,"['Screen Gems', '2.0 Entertainment', 'Jesus & ...",65675816,103
1,640146,Ant-Man and the Wasp: Quantumania,"['Action', 'Adventure', 'Science Fiction']",English,6.6,2294,4665.438,200000000,"['Marvel Studios', 'Kevin Feige Productions']",464566092,125
2,502356,The Super Mario Bros. Movie,"['Animation', 'Adventure', 'Family', 'Fantasy'...",English,7.5,1861,3935.550,100000000,"['Universal Pictures', 'Illumination', 'Ninten...",1121048165,92
3,868759,Ghosted,"['Action', 'Comedy', 'Romance']",English,7.2,652,2791.532,0,"['Skydance Media', 'Apple Studios']",0,120
4,594767,Shazam! Fury of the Gods,"['Action', 'Comedy', 'Fantasy', 'Adventure']",English,6.8,1510,2702.593,125000000,"['New Line Cinema', 'The Safran Company', 'DC ...",133437105,130


In [8]:
df.shape

(10000, 11)

In [9]:
df.genres.unique()

array(["['Horror', 'Mystery', 'Thriller']",
       "['Action', 'Adventure', 'Science Fiction']",
       "['Animation', 'Adventure', 'Family', 'Fantasy', 'Comedy']", ...,
       "['Thriller', 'Drama', 'Music']",
       "['Fantasy', 'Animation', 'Action', 'Adventure', 'Science Fiction', 'Drama', 'Romance']",
       "['TV Movie', 'Fantasy', 'Animation', 'Action', 'Thriller', 'Science Fiction', 'Horror']"],
      dtype=object)

In [10]:
data = df.copy()

In [19]:
# User input
user_genres = ['Action', 'Adventure', 'Sci-Fi']  # Example user input

# Fit a clustering algorithm (K-means) to the genre matrix
num_clusters = 5  # Adjust as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(genre_matrix)

# Calculate cluster centroids
cluster_centroids = kmeans.cluster_centers_

# Transform user genres using the MultiLabelBinarizer
user_profile = mlb.transform([user_genres])

# Assign user profile to the nearest cluster centroid
user_cluster = pairwise_distances_argmin_min(user_profile, cluster_centroids)[0][0]

# Get movie recommendations from the same cluster
cluster_movies = data[cluster_labels == user_cluster]

# Calculate similarity scores between user profile and cluster movies
cluster_genre_matrix = mlb.transform(cluster_movies['genres'].apply(lambda x: x.split('|')))
similarity_scores = cosine_similarity(user_profile, cluster_genre_matrix)

# Sort movies by popularity or ratings (you can adjust this based on your data)
cluster_movies['similarity_score'] = similarity_scores[0]
cluster_movies = cluster_movies.sort_values(by='popularity', ascending=False)

# Display recommendations along with similarity scores
num_recommendations = 10
if num_recommendations > len(cluster_movies):
    num_recommendations = len(cluster_movies)

recommendations = cluster_movies.head(num_recommendations)
for idx, movie in recommendations.iterrows():
    print(f"Title: {movie['title']}")
    print(f"Genres: {movie['genres']}")
    print(f"Similarity Score: {movie['similarity_score']:.2f}")
    print("---")

C:\Users\Richard\Anaconda june 2023\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Title: The Pope's Exorcist
Genres: ['Horror', 'Mystery', 'Thriller']
Similarity Score: 0.00
---
Title: Ant-Man and the Wasp: Quantumania
Genres: ['Action', 'Adventure', 'Science Fiction']
Similarity Score: 0.00
---
Title: The Super Mario Bros. Movie
Genres: ['Animation', 'Adventure', 'Family', 'Fantasy', 'Comedy']
Similarity Score: 0.00
---
Title: Ghosted
Genres: ['Action', 'Comedy', 'Romance']
Similarity Score: 0.00
---
Title: Shazam! Fury of the Gods
Genres: ['Action', 'Comedy', 'Fantasy', 'Adventure']
Similarity Score: 0.00
---
Title: Guardians of the Galaxy Volume 3
Genres: ['Science Fiction', 'Adventure', 'Action']
Similarity Score: 0.00
---
Title: Avatar: The Way of Water
Genres: ['Science Fiction', 'Adventure', 'Action']
Similarity Score: 0.00
---
Title: Creed III
Genres: ['Drama', 'Action']
Similarity Score: 0.00
---
Title: Scream VI
Genres: ['Horror', 'Mystery', 'Thriller']
Similarity Score: 0.00
---
Title: Dungeons & Dragons: Honor Among Thieves
Genres: ['Adventure', 'Fantasy

C:\Users\Richard\AppData\Local\Temp\ipykernel_25880\2129752797.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_movies['similarity_score'] = similarity_scores[0]


In [20]:
df.isnull().sum()

id                      0
title                   0
genres                  0
original_language       0
vote_average            0
vote_count              0
popularity              0
budget                  0
production_companies    0
revenue                 0
runtime                 0
dtype: int64

In [21]:
df.head()

,id,title,genres,original_language,vote_average,vote_count,popularity,budget,production_companies,revenue,runtime
0,758323,The Pope's Exorcist,"['Horror', 'Mystery', 'Thriller']",English,7.4,619,5089.969,18000000,"['Screen Gems', '2.0 Entertainment', 'Jesus & ...",65675816,103
1,640146,Ant-Man and the Wasp: Quantumania,"['Action', 'Adventure', 'Science Fiction']",English,6.6,2294,4665.438,200000000,"['Marvel Studios', 'Kevin Feige Productions']",464566092,125
2,502356,The Super Mario Bros. Movie,"['Animation', 'Adventure', 'Family', 'Fantasy'...",English,7.5,1861,3935.550,100000000,"['Universal Pictures', 'Illumination', 'Ninten...",1121048165,92
3,868759,Ghosted,"['Action', 'Comedy', 'Romance']",English,7.2,652,2791.532,0,"['Skydance Media', 'Apple Studios']",0,120
4,594767,Shazam! Fury of the Gods,"['Action', 'Comedy', 'Fantasy', 'Adventure']",English,6.8,1510,2702.593,125000000,"['New Line Cinema', 'The Safran Company', 'DC ...",133437105,130


In [22]:
df.corr()

C:\Users\Richard\AppData\Local\Temp\ipykernel_25880\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,id,vote_average,vote_count,popularity,budget,revenue,runtime
id,1.000000,-0.241569,-0.259859,0.102688,-0.243411,-0.207591,-0.256838
vote_average,-0.241569,1.000000,0.253543,0.040162,0.074849,0.149643,0.388440
vote_count,-0.259859,0.253543,1.000000,0.069693,0.600121,0.753206,0.288462
popularity,0.102688,0.040162,0.069693,1.000000,0.143257,0.148195,0.038973
budget,-0.243411,0.074849,0.600121,0.143257,1.000000,0.735239,0.282498
revenue,-0.207591,0.149643,0.753206,0.148195,0.735239,1.000000,0.253162
runtime,-0.256838,0.388440,0.288462,0.038973,0.282498,0.253162,1.000000
